In [5]:
import os
import sys
import dspy
import rich
import polars as pl
from pathlib import Path

## Load the Full Pathology Report Dataset

In [7]:
df_path_reports = pl.read_csv("../../data/tcga/TCGA_reports.csv")
assert len(df_path_reports) == 9523
df_path_reports.head(10)

patient_filename,text
str,str
"""TCGA-BP-5195.25c0b433-5557-416…","""Date of Recelpt: Clinical Diag…"
"""TCGA-D7-8573.b7306a47-697d-4ed…","""Material: 1) Material: stomach…"
"""TCGA-EI-7004.13591eed-30e5-47a…","""page 1 / 1. copy No. 3. Examin…"
"""TCGA-EB-A82B.23E186C6-739C-4EF…","""Patient ID: Gross Description:…"
"""TCGA-A6-3808.e1505f65-72ef-438…","""SPECIMEN. Right colon. CLINICA…"
"""TCGA-29-1761.63defa27-e729-451…","""CLINICAL HISTORY: Pelvic organ…"
"""TCGA-24-1616.d8c764d9-0fe8-488…","""Address: : Physician (s) : Oth…"
"""TCGA-IN-A6RO.C0E5F045-3F12-45C…","""FINAL DIAGNOSIS: PART 1: ESOPH…"
"""TCGA-44-8119.c40ea4c8-8cbe-4a4…","""Final Surgical Pathology Repor…"


In [10]:
report = df_path_reports.select("text").to_series().to_list()[0]
rich.print(report)

Date of Recelpt: Clinical Diagnosis & History: Incidental 3 cm left upper pole renal mass. Specimens Submitted: 1: 
Kidney, Left Upper Pole; Partial Nephrectomy. DIAGNOSIS: 1. Kidney, Left Upper Pole; Partial Nephrectomy: Tumor 
Type: Renal cell carcinoma - Conventional (clear cell) type. Fuhrman Nuclear Grade: Nuclear grade II/IV. Tumor 
Size: Greatest diameter is 2.4 cm. Local Invasion (for renal cortical types): Not Identified. Renal Vein Invasion: 
Not identified. Surgical Margins: Free of tumor. Non-Neoplastic Kidney: shows focal chronic inflammation and focal 
superficial glomerulosclerosis. Adrenal Gland: Not identified. Lymph Nodes: Not identified. Staging for renal cell 
carcinoma/oncocytoma: pT1 Tumor <= 7.0 cm in greatest dimension limited to the kidney. JATTEST THAT THE ABOVE 
DIAGNOSIS IS BASED UPON MY PERSONAL EXAMINATION OF THE SLIDES (AND/OR OTHER MATERIAL), AND THAT IHAVE. REVIEWED AND
APPROVED THIS REPORT. Gross Description: 1). The specimen Is received fresh for frozen section consultation, 
labeled "left upper pole renal tumor stitch marks deep margin". It consists of a 6.6 X 3.7 x 2.9 cm wedge shaped 
portion of kidney with a suture marking the deep margin and attached 12.2 x 8,5 x. 2.7 cm of perirenal. Also 
received in the same container Is a separate 11.6 x 8.4 x 4.6 cm aggregate of unremarkable. fibroadipose tissue. 
The margin is inked black and the specimen is serially sectioned to reveal a well-circumscribed cortically. based 
yellow focally hemorrhagic 2.4 x 2.0 x 1.9 cm tumor. The tumor appears grossly confined to the renal parenchyma, 
no. extension into perirenal fat is identified. The clearance from the resection margin is 1.1 cm. A representative
section of the. nearest margin is submilted for frozen section diagnosis. Representatively submitted. A photograph 
Is taken. Portions of the. tumor are submitted for TPS. Summary of sections: FSC - frozen section control. T tumor.
M margin. RS - representative sections. Summary of Sections: Part 1: Kidney, Left Upper Pole; Partial Nephrectomy. 
Block. Sect. Site. PCs. 1. fsc. 1. 2. m. 2. 1. rs. 1. 2. t. 2. Intraoperative Consultation: Note: The diagnoses 
given in this section pertain only to the. tissue sample examined at the time of the intraoperative. consultation. 
1). FROZEN SECTION DIAGNOSIS: LEFT UPPER POLE RENAL TUMOR. RENAL CORTICAL NEOPLASM. WITH BENIGN MARGIN. PERMANENT 
DIAGNOSIS: SAME.

## Load the Annotated Dataset

In [11]:
df_annotated = pl.read_csv(
    "../../tcga-path-reports/data/tcga_metadata/tcga_patient_to_cancer_type.csv"
)
df_annotated.head(10)

patient_id,cancer_type
str,str
"""TCGA-OR-A5J1""","""ACC"""
"""TCGA-OR-A5J2""","""ACC"""
"""TCGA-OR-A5J3""","""ACC"""
"""TCGA-OR-A5J4""","""ACC"""
"""TCGA-OR-A5J5""","""ACC"""
"""TCGA-OR-A5J6""","""ACC"""
"""TCGA-OR-A5J7""","""ACC"""
"""TCGA-OR-A5J8""","""ACC"""
"""TCGA-OR-A5J9""","""ACC"""


In [12]:
path_text = df_path_reports.filter(
    pl.col("patient_filename").str.contains("TCGA-OR-A5J1")
)["text"][0]
cancer_type = df_annotated.filter(pl.col("patient_id") == "TCGA-OR-A5J1")[
    "cancer_type"
][0]
print(cancer_type)
rich.print(path_text)

ACC


Deliver To: NOS P. Cuncertain inknown behavior 8370/1 A. SPECIMEN TYPE: Adrenal. Blopsy No: T. CLINICAL NOTES: H. 
Histopathology of L) adrenal gland. Conn's syndrome (note small section of. O. tumour excised for genetic studies).
L. MACROSCOPIC: o. "Left adrenal. The specimen consists of an adrenal gland with a post fixation. weight 102.4g. It
measures 110 x 70 x 60mm. A section of tissue has been. G. taken from one pole. On sectioning, a nodular lesion 
with a maximum diameter. Y. of 25 x 30mm, and extending for a length of 70mm, can be identified. The cut. surface 
of this lesion is irregular. it can be seen to be compressing the. adjacent normal adrenal tissue. Block 1: tissue 
near the excision where previous sampling for genetic studies. has already been performed. Blocks 2 and 3: lesion 
where it abuts the normal adrenal gland. Block 4: lesion with its closest proximity to the excision margin. Blocks 
5-9: further blocks of lesion. MICROSCOPIC: Sections show an adrenal tumour with a predominantly trabecular 
architectural. A. pattern and demonstrating a mitotic rate of <6 per 50HPF. Cytological atypia is. N. present. No 
absolutely definite vascular invasion is seen. The tumour appears. close but clear of the painted excision margin. 
A. The morphological appearances of this adrenal tumour in association with the. T. results of immunoperoxidase 
stains (ie. positive for vimentin and negative for. Cam 5.2 and chromogranin) are consistent with a cortical 
adrenal tumour, which. o. is considered best regarded as an "adrenal cortical neoplasm of indeterminate. M. 
malignant potential", and continuing long-term clinical follow-up Is strongly. recommended. I. SUMMARY: Considered 
best regarded as an "adrenal cortical tumour of. C. indeterminate malignant potential". A. Note: Please see 
attached references from Tumours of the Adrenal Gland and. L. Extra-Adrenal Paraganglioma by Ernest E.Lack: No 19 -
3rd series; from the. Armed Forces institute of Pathology, Washington D.C. P. T-93000 M-09350. A. ADDITIONAL 
REPORT: T. ELECTRON MICROSCOPY: (received from. H. O. L. Page 1 of. O. Requesting Doctor's Information: Deliver To:
A. SPECIMEN TYPE: Adrenal. Biopsy No: T. H. "The majority of cells examined by electron microscopy show the 
features of. aldosterone-secreting tumours. In particular, mitochondria elongated with. o. numerous lamellar 
cristae ("sarcotubular" mitochondria), similar to those of. L. the normal zona glomerulosa. However, spherical 
"hybrid" mitochondria with. lamellar or tubular cristae are also seen. Rough and smooth surfaced. o. endoplasmic 
reticulum are present in moderate amounts. Lipid droplets are. G. sparse. A few pleomorphic nuclei were seen, but 
the majority appear spherical. with a regular chromatin pattern and a small nucleolus. Y. Most authorities agree 
that it is not possible to predict the likely behaviour. of adrenal cortical tumours from their ultrastructural 
appearances. Reliance. should be placed on the usual histological and macroscopic criteria.". REPORTING 
PATHOLOGIST: A. (Electronic Signature). N. A. T. o. M. I. C. A. L. P. A. T. H. O. L. O.

## DSPy it

In [13]:
lm = dspy.LM("openai/gpt-4o-mini", api_key=os.environ["OPENAI_API_KEY"])
dspy.configure(lm=lm)

In [14]:
class ExtractInfo(dspy.Signature):
    """Extract structured information from text."""

    text: str = dspy.InputField()
    cancer_type: str = dspy.OutputField(
        desc="the type of primary cancer diagnosis in the pathology report text"
    )


module = dspy.Predict(ExtractInfo)
response = module(text=path_text)

print(response.cancer_type)

adrenal cortical neoplasm of indeterminate malignant potential
